# Save and Store Features
In this notebook we will compute all prediction and store the relative features in drive using the model computed in the notebook "ResNet50".

*Note*: the features related to the simple feature extraction model are already computed in the notebook "ResNet50", thus they won't again be computed here.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import tensorflow as tf
from tensorflow import keras as ks
from tensorflow.keras import layers
from tensorflow.keras.applications import ResNet50V2
from tensorflow.keras import regularizers
import pathlib
import matplotlib.pyplot as plt
import numpy as np

In [3]:
! pip install -q kaggle

from google.colab import files
_ = files.upload()

! mkdir -p ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json


In [4]:
! kaggle datasets download -d gpiosenka/100-bird-species

 99% 1.16G/1.17G [00:11<00:00, 148MB/s]
100% 1.17G/1.17G [00:11<00:00, 106MB/s]


In [5]:
!unzip 100-bird-species.zip

Output streaming troncato alle ultime 5000 righe.
  inflating: train/TURQUOISE MOTMOT/043.jpg  
  inflating: train/TURQUOISE MOTMOT/044.jpg  
  inflating: train/TURQUOISE MOTMOT/045.jpg  
  inflating: train/TURQUOISE MOTMOT/046.jpg  
  inflating: train/TURQUOISE MOTMOT/047.jpg  
  inflating: train/TURQUOISE MOTMOT/048.jpg  
  inflating: train/TURQUOISE MOTMOT/049.jpg  
  inflating: train/TURQUOISE MOTMOT/050.jpg  
  inflating: train/TURQUOISE MOTMOT/051.jpg  
  inflating: train/TURQUOISE MOTMOT/052.jpg  
  inflating: train/TURQUOISE MOTMOT/053.jpg  
  inflating: train/TURQUOISE MOTMOT/054.jpg  
  inflating: train/TURQUOISE MOTMOT/055.jpg  
  inflating: train/TURQUOISE MOTMOT/056.jpg  
  inflating: train/TURQUOISE MOTMOT/057.jpg  
  inflating: train/TURQUOISE MOTMOT/058.jpg  
  inflating: train/TURQUOISE MOTMOT/059.jpg  
  inflating: train/TURQUOISE MOTMOT/060.jpg  
  inflating: train/TURQUOISE MOTMOT/061.jpg  
  inflating: train/TURQUOISE MOTMOT/062.jpg  
  inflating: train/TURQUOISE M

## Create the different sets
In this section the training set, the test set and the discrimator sets are computed in order to extract the features from them

In [6]:
TRAIN_DIR = 'train/'
VALID_DIR = 'valid/'
TEST_DIR = 'test/'
DISTRACTOR_DIR = 'mirflickr'
BATCH_SIZE = 128
IMAGE_HEIGHT = 224
IMAGE_WIDTH = 224
RANDOM_SEED = 42

Distractor path:

In [7]:
!unzip -q '/content/drive/My Drive/CV_Birds/mirflickr.zip' -d '/content'

Create sets:

In [ ]:
training_images = tf.keras.preprocessing.image_dataset_from_directory(
    TRAIN_DIR, labels='inferred', label_mode='categorical',
    class_names=None, color_mode='rgb', batch_size=BATCH_SIZE, image_size=(IMAGE_HEIGHT,
    IMAGE_WIDTH), shuffle=False, seed=RANDOM_SEED, interpolation='bilinear')

test_images = tf.keras.preprocessing.image_dataset_from_directory(
    TEST_DIR, labels='inferred', label_mode='categorical',
    class_names=None, color_mode='rgb', batch_size=BATCH_SIZE, image_size=(IMAGE_HEIGHT,
    IMAGE_WIDTH), shuffle=False, seed=RANDOM_SEED, interpolation='bilinear')

Found 47332 files belonging to 325 classes.
Found 1625 files belonging to 325 classes.


In [8]:
distractor_images = tf.keras.preprocessing.image_dataset_from_directory(
    DISTRACTOR_DIR,
    image_size = (IMAGE_HEIGHT, IMAGE_WIDTH),
    batch_size = BATCH_SIZE,
    seed=RANDOM_SEED,
    labels=None,
    label_mode=None)

Found 25000 files belonging to 1 classes.


## Model 1
Load the model from drive:

In [18]:
MODEL_PATH = '/content/drive/MyDrive/CV_Birds/models/ResNet50v2/model1.keras'
model = ks.models.load_model(MODEL_PATH)

In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, None, None, 3)]   0         
                                                                 
 resnet50v2 (Functional)     (None, 2048)              23564800  
                                                                 
 predictions (Dense)         (None, 325)               665925    
                                                                 
Total params: 24,230,725
Trainable params: 16,755,013
Non-trainable params: 7,475,712
_________________________________________________________________


In [19]:
a = ks.models.Sequential(model.layers[:2])

In [ ]:
a.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50v2 (Functional)     (None, 2048)              23564800  
                                                                 
Total params: 23,564,800
Trainable params: 16,089,088
Non-trainable params: 7,475,712
_________________________________________________________________


Predict features for training set and save them:

In [ ]:
features_model = a.predict(training_images, batch_size=BATCH_SIZE, verbose=True)

370/370 [==============================] - 261s 672ms/step


In [ ]:
np.save('/content/drive/MyDrive/CV_Birds/features/training/ResNet50v2/model1_train_features.npy', features_model)

Predict features for test set and save them:

In [ ]:
features_model = a.predict(test_images, batch_size=BATCH_SIZE, verbose=True)

13/13 [==============================] - 11s 937ms/step


In [ ]:
np.save('/content/drive/MyDrive/CV_Birds/features/test/ResNet50v2/model1_test_features.npy', features_model)

Predict features for the distractor and save them

In [20]:
features_model = a.predict(distractor_images, batch_size=BATCH_SIZE, verbose=True)

196/196 [==============================] - 128s 631ms/step


In [21]:
np.save('/content/drive/MyDrive/CV_Birds/features/distractor/ResNet50v2/model1_distractor_features.npy', features_model)

## Model 2
Load the model from drive:

In [22]:
MODEL_PATH = '/content/drive/MyDrive/CV_Birds/models/ResNet50v2/model2.keras'
model = ks.models.load_model(MODEL_PATH)

In [14]:
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, None, None, 3)]   0         
                                                                 
 resnet50v2 (Functional)     (None, 2048)              23564800  
                                                                 
 predictions (Dense)         (None, 325)               665925    
                                                                 
Total params: 24,230,725
Trainable params: 15,636,805
Non-trainable params: 8,593,920
_________________________________________________________________


In [23]:
a = ks.models.Sequential(model.layers[:2])

In [16]:
a.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50v2 (Functional)     (None, 2048)              23564800  
                                                                 
Total params: 23,564,800
Trainable params: 14,970,880
Non-trainable params: 8,593,920
_________________________________________________________________


Predict features for training set and save them:

In [ ]:
features_model = a.predict(training_images, batch_size=BATCH_SIZE, verbose=True)

370/370 [==============================] - 246s 665ms/step


In [ ]:
np.save('/content/drive/MyDrive/CV_Birds/features/training/ResNet50v2/model2_train_features.npy', features_model)

Predict features for test set and save them:

In [ ]:
features_model = a.predict(test_images, batch_size=BATCH_SIZE, verbose=True)

13/13 [==============================] - 8s 661ms/step


In [ ]:
np.save('/content/drive/MyDrive/CV_Birds/features/test/ResNet50v2/model2_test_features.npy', features_model)

Predict features for the distractor and save them

In [24]:
features_model = a.predict(distractor_images, batch_size=BATCH_SIZE, verbose=True)

196/196 [==============================] - 128s 634ms/step


In [25]:
np.save('/content/drive/MyDrive/CV_Birds/features/distractor/ResNet50v2/model2_distractor_features.npy', features_model)

## Model 3
Load the model from drive:

In [26]:
MODEL_PATH = '/content/drive/MyDrive/CV_Birds/models/ResNet50v2/model3.keras'
model = ks.models.load_model(MODEL_PATH)

In [27]:
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, None, None, 3)]   0         
                                                                 
 resnet50v2 (Functional)     (None, 2048)              23564800  
                                                                 
 predictions (Dense)         (None, 325)               665925    
                                                                 
Total params: 24,230,725
Trainable params: 5,134,661
Non-trainable params: 19,096,064
_________________________________________________________________


In [28]:
a = ks.models.Sequential(model.layers[:2])

In [29]:
a.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50v2 (Functional)     (None, 2048)              23564800  
                                                                 
Total params: 23,564,800
Trainable params: 4,468,736
Non-trainable params: 19,096,064
_________________________________________________________________


Predict features for training set and save them:

In [ ]:
features_model = a.predict(training_images, batch_size=BATCH_SIZE, verbose=True)

370/370 [==============================] - 246s 664ms/step


In [ ]:
np.save('/content/drive/MyDrive/CV_Birds/features/training/ResNet50v2/model3_train_features.npy', features_model)

Predict features for test set and save them:

In [ ]:
features_model = a.predict(test_images, batch_size=BATCH_SIZE, verbose=True)

13/13 [==============================] - 8s 661ms/step


In [ ]:
np.save('/content/drive/MyDrive/CV_Birds/features/test/ResNet50v2/model3_test_features.npy', features_model)

Predict features for the distractor and save them

In [30]:
features_model = a.predict(distractor_images, batch_size=BATCH_SIZE, verbose=True)

196/196 [==============================] - 129s 635ms/step


In [31]:
np.save('/content/drive/MyDrive/CV_Birds/features/distractor/ResNet50v2/model3_distractor_features.npy', features_model)

## Model 4
Load the model from drive:

In [32]:
MODEL_PATH = '/content/drive/MyDrive/CV_Birds/models/ResNet50v2/model4.keras'
model = ks.models.load_model(MODEL_PATH)

In [33]:
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, None, None, 3)]   0         
                                                                 
 resnet50v2 (Functional)     (None, 2048)              23564800  
                                                                 
 predictions (Dense)         (None, 325)               665925    
                                                                 
Total params: 24,230,725
Trainable params: 9,599,301
Non-trainable params: 14,631,424
_________________________________________________________________


In [34]:
a = ks.models.Sequential(model.layers[:2])

In [35]:
a.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50v2 (Functional)     (None, 2048)              23564800  
                                                                 
Total params: 23,564,800
Trainable params: 8,933,376
Non-trainable params: 14,631,424
_________________________________________________________________


Predict features for training set and save them:

In [ ]:
features_model = a.predict(training_images, batch_size=BATCH_SIZE, verbose=True)

370/370 [==============================] - 246s 664ms/step


In [ ]:
np.save('/content/drive/MyDrive/CV_Birds/features/training/ResNet50v2/model4_train_features.npy', features_model)

Predict features for test set and save them:

In [ ]:
features_model = a.predict(test_images, batch_size=BATCH_SIZE, verbose=True)

13/13 [==============================] - 8s 662ms/step


In [ ]:
np.save('/content/drive/MyDrive/CV_Birds/features/test/ResNet50v2/model4_test_features.npy', features_model)

Predict features for the distractor and save them

In [36]:
features_model = a.predict(distractor_images, batch_size=BATCH_SIZE, verbose=True)

196/196 [==============================] - 128s 632ms/step


In [37]:
np.save('/content/drive/MyDrive/CV_Birds/features/distractor/ResNet50v2/model4_distractor_features.npy', features_model)

## Model 9
Load the model from drive:

In [38]:
MODEL_PATH = '/content/drive/MyDrive/CV_Birds/models/ResNet50v2/model9.keras'
model = ks.models.load_model(MODEL_PATH)

In [39]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, None, None, 3)]   0         
                                                                 
 resnet50v2 (Functional)     (None, 2048)              23564800  
                                                                 
 predictions (Dense)         (None, 325)               665925    
                                                                 
Total params: 24,230,725
Trainable params: 17,873,221
Non-trainable params: 6,357,504
_________________________________________________________________


In [40]:
a = ks.models.Sequential(model.layers[:2])

In [41]:
a.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50v2 (Functional)     (None, 2048)              23564800  
                                                                 
Total params: 23,564,800
Trainable params: 17,207,296
Non-trainable params: 6,357,504
_________________________________________________________________


Predict features for training set and save them:

In [ ]:
features_model = a.predict(training_images, batch_size=BATCH_SIZE, verbose=True)

370/370 [==============================] - 247s 667ms/step


In [ ]:
np.save('/content/drive/MyDrive/CV_Birds/features/training/ResNet50v2/model9_train_features.npy', features_model)

Predict features for test set and save them:

In [ ]:
features_model = a.predict(test_images, batch_size=BATCH_SIZE, verbose=True)

13/13 [==============================] - 8s 663ms/step


In [ ]:
np.save('/content/drive/MyDrive/CV_Birds/features/test/ResNet50v2/model9_test_features.npy', features_model)

Predict features for the distractor and save them

In [42]:
features_model = a.predict(distractor_images, batch_size=BATCH_SIZE, verbose=True)

196/196 [==============================] - 128s 633ms/step


In [43]:
np.save('/content/drive/MyDrive/CV_Birds/features/distractor/ResNet50v2/model9_distractor_features.npy', features_model)

## Model 10
Load the model from drive:

In [44]:
MODEL_PATH = '/content/drive/MyDrive/CV_Birds/models/ResNet50v2/model10.keras'
model = ks.models.load_model(MODEL_PATH)

In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, None, None, 3)]   0         
                                                                 
 resnet50v2 (Functional)     (None, 2048)              23564800  
                                                                 
 predictions (Dense)         (None, 325)               665925    
                                                                 
Total params: 24,230,725
Trainable params: 18,991,429
Non-trainable params: 5,239,296
_________________________________________________________________


In [45]:
a = ks.models.Sequential(model.layers[:2])

In [ ]:
a.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50v2 (Functional)     (None, 2048)              23564800  
                                                                 
Total params: 23,564,800
Trainable params: 18,325,504
Non-trainable params: 5,239,296
_________________________________________________________________


Predict features for training set and save them:

In [ ]:
features_model = a.predict(training_images, batch_size=BATCH_SIZE, verbose=True)

370/370 [==============================] - 245s 661ms/step


In [ ]:
np.save('/content/drive/MyDrive/CV_Birds/features/training/ResNet50v2/model10_train_features.npy', features_model)

Predict features for test set and save them:

In [ ]:
features_model = a.predict(test_images, batch_size=BATCH_SIZE, verbose=True)

13/13 [==============================] - 8s 657ms/step


In [ ]:
np.save('/content/drive/MyDrive/CV_Birds/features/test/ResNet50v2/model10_test_features.npy', features_model)

Predict features for the distractor and save them

In [46]:
features_model = a.predict(distractor_images, batch_size=BATCH_SIZE, verbose=True)

196/196 [==============================] - 129s 635ms/step


In [47]:
np.save('/content/drive/MyDrive/CV_Birds/features/distractor/ResNet50v2/model10_distractor_features.npy', features_model)

## Model 11
Load the model from drive:

In [48]:
MODEL_PATH = '/content/drive/MyDrive/CV_Birds/models/ResNet50v2/model11.keras'
model = ks.models.load_model(MODEL_PATH)

In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, None, None, 3)]   0         
                                                                 
 resnet50v2 (Functional)     (None, 2048)              23564800  
                                                                 
 predictions (Dense)         (None, 325)               665925    
                                                                 
Total params: 24,230,725
Trainable params: 20,109,637
Non-trainable params: 4,121,088
_________________________________________________________________


In [49]:
a = ks.models.Sequential(model.layers[:2])

In [ ]:
a.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50v2 (Functional)     (None, 2048)              23564800  
                                                                 
Total params: 23,564,800
Trainable params: 19,443,712
Non-trainable params: 4,121,088
_________________________________________________________________


Predict features for training set and save them:

In [ ]:
features_model = a.predict(training_images, batch_size=BATCH_SIZE, verbose=True)

370/370 [==============================] - 245s 660ms/step


In [ ]:
np.save('/content/drive/MyDrive/CV_Birds/features/training/ResNet50v2/model11_train_features.npy', features_model)

Predict features for test set and save them:

In [ ]:
features_model = a.predict(test_images, batch_size=BATCH_SIZE, verbose=True)

13/13 [==============================] - 8s 657ms/step


In [ ]:
np.save('/content/drive/MyDrive/CV_Birds/features/test/ResNet50v2/model11_test_features.npy', features_model)

Predict features for the distractor and save them

In [50]:
features_model = a.predict(distractor_images, batch_size=BATCH_SIZE, verbose=True)

196/196 [==============================] - 128s 633ms/step


In [51]:
np.save('/content/drive/MyDrive/CV_Birds/features/distractor/ResNet50v2/model11_distractor_features.npy', features_model)

# Resnet 101

## Model 1
Load the model from drive:

In [52]:
MODEL_PATH = '/content/drive/MyDrive/CV_Birds/models/ResNet101v2/resNet101_model1.keras'
model = ks.models.load_model(MODEL_PATH)

In [ ]:
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, None, None, 3)]   0         
                                                                 
 resnet101v2 (Functional)    (None, 2048)              42626560  
                                                                 
 predictions (Dense)         (None, 325)               665925    
                                                                 
Total params: 43,292,485
Trainable params: 20,109,637
Non-trainable params: 23,182,848
_________________________________________________________________


In [53]:
a = ks.models.Sequential(model.layers[:2])

In [ ]:
a.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet101v2 (Functional)    (None, 2048)              42626560  
                                                                 
Total params: 42,626,560
Trainable params: 19,443,712
Non-trainable params: 23,182,848
_________________________________________________________________


Predict features for training set and save them:

In [ ]:
features_model = a.predict(training_images, batch_size=BATCH_SIZE, verbose=True)

370/370 [==============================] - 457s 1s/step


In [ ]:
np.save('/content/drive/MyDrive/CV_Birds/features/training/ResNet101v2/model1_train_features.npy', features_model)

Predict features for test set and save them:

In [ ]:
features_model = a.predict(test_images, batch_size=BATCH_SIZE, verbose=True)

13/13 [==============================] - 18s 1s/step


In [ ]:
np.save('/content/drive/MyDrive/CV_Birds/features/test/ResNet101v2/model1_test_features.npy', features_model)

Predict features for the distractor and save them

In [54]:
features_model = a.predict(distractor_images, batch_size=BATCH_SIZE, verbose=True)

196/196 [==============================] - 224s 1s/step


In [55]:
np.save('/content/drive/MyDrive/CV_Birds/features/distractor/ResNet101v2/model1_distractor_features.npy', features_model)

## Model 2
Load the model from drive:

In [56]:
MODEL_PATH = '/content/drive/MyDrive/CV_Birds/models/ResNet101v2/resNet101_model2.keras'
model = ks.models.load_model(MODEL_PATH)

In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, None, None, 3)]   0         
                                                                 
 resnet101v2 (Functional)    (None, 2048)              42626560  
                                                                 
 predictions (Dense)         (None, 325)               665925    
                                                                 
Total params: 43,292,485
Trainable params: 30,173,509
Non-trainable params: 13,118,976
_________________________________________________________________


In [57]:
a = ks.models.Sequential(model.layers[:2])

In [ ]:
a.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet101v2 (Functional)    (None, 2048)              42626560  
                                                                 
Total params: 42,626,560
Trainable params: 29,507,584
Non-trainable params: 13,118,976
_________________________________________________________________


Predict features for training set and save them:

In [ ]:
features_model = a.predict(training_images, batch_size=BATCH_SIZE, verbose=True)

370/370 [==============================] - 444s 1s/step


In [ ]:
np.save('/content/drive/MyDrive/CV_Birds/features/training/ResNet101v2/model2_train_features.npy', features_model)

Predict features for test set and save them:

In [ ]:
features_model = a.predict(test_images, batch_size=BATCH_SIZE, verbose=True)

13/13 [==============================] - 15s 1s/step


In [ ]:
np.save('/content/drive/MyDrive/CV_Birds/features/test/ResNet101v2/model2_test_features.npy', features_model)

Predict features for the distractor and save them

In [58]:
features_model = a.predict(distractor_images, batch_size=BATCH_SIZE, verbose=True)

196/196 [==============================] - 223s 1s/step


In [59]:
np.save('/content/drive/MyDrive/CV_Birds/features/distractor/ResNet101v2/model2_distractor_features.npy', features_model)